### Analyze user trip and user label

In [1]:
import csv 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

In [14]:
stationData = open("../../Metro/station_transInfo_cleaned.csv")
stationLoc = {}
next(stationData)
for row in stationData:
    row = row.rstrip().split(',')
    # stationID = int(row[0])
    # lon = float(row[1])
    # lat = float(row[2])
    stationID = int(row[1])
    lon = float(row[2])
    lat = float(row[3])
    stationLoc[stationID] = (lon, lat)

# new tranfer station ids
transferStation, transferStationToNewID = pickle.load(open("../../Metro/transferStations.pkl", 'rb'))

# inFile = 'results/metroData_filtered.csv'
inFile = '/data6/peiyan/SH-METR/results/metroData_userTrip.csv'
userData = open(inFile, 'r')

show numRecords

In [15]:
numRecords = []
trips = []
preUser = ''
userDataList=list(userData)
for line in tqdm(userDataList):
    line = line.rstrip().split(',')
    if len(line) == 1:
        # a new user
        currentUser = int(line[0])
        if preUser == '':
            preUser = currentUser

        # process the last user
        if currentUser != preUser:
            # filter users
            numRecords.append(len(trips))
            # initialize
            preUser = currentUser
            trips = []
        continue

    currentDay = line[0]
    currentTime = line[1]
    inStation = int(line[2])
    outStation = int(line[3])

    # try:
    #     inStation_new = transferStationToNewID[inStation]
    # except:
    #     inStation_new = inStation
    # try:
    #     outStation_new = transferStationToNewID[outStation]
    # except:
    #     outStation_new = outStation

    # if inStation_new == outStation_new:
    #     continue

    # if inStation_new not in stationLoc or outStation_new not in stationLoc:
    #     continue

    trips.append([currentDay, currentTime, inStation, outStation])  # with updated stationName

userData.close()

100%|██████████| 756544984/756544984 [16:36<00:00, 759256.36it/s]  


In [2]:
 # load
userLabels, stationLabels, tripLabels = pickle.load(open("/data6/peiyan/SH-METR/results/userLabels.pkl", 'rb'))

numCommuters = np.sum([i for i in userLabels.values() if i==1])
numNoHome = np.sum([-i for i in userLabels.values() if i==-1])
numNonCommuters = len(userLabels) - numCommuters - numNoHome

# print "# of commuters : ", numCommuters/float(numNonCommuters + numCommuters)
# print "# of non-commuters : ", numNonCommuters/float(numNonCommuters + numCommuters)
# print "# of no home : ", numNoHome/float(len(userLabels))
print("# of commuters: {:.4f}".format(numCommuters/(numNonCommuters + numCommuters)))
print("# of non-commuters: {:.4f}".format(numNonCommuters/(numNonCommuters + numCommuters)))
print("# of no home: {:.4f}".format(numNoHome/len(userLabels)))

# # distribution of trip distances for all, commuters, non-commuters
# distances = pickle.load(open("results/stationDistances.pkl", 'rb'))

# allTripsDistances = []
# distancesInUserTypes = [[] for i in range(2)]
# distancesInTripTypes = [[] for i in range(4)]

# for user in userLabels:
#     label = userLabels[user]
# for trip, tripL in tripLabels[user].iteritems():
#     dist = distances[trip]
#     allTripsDistances.append(dist)
#     distancesInUserTypes[label].append(dist)
#     distancesInTripTypes[tripL].append(dist)

# # plot the distritbuion
# print (np.max(allTripsDistances))

# # travel distance of varying types of users
# numInterval = 100
# maxV = 100.0
# interval = maxV / numInterval
# bins = np.linspace(0, maxV, numInterval + 1)


# usagesHist = np.histogram(np.array(allTripsDistances), bins)
# usagesHist_all = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # non-commuter
# usagesHist = np.histogram(np.array(distancesInUserTypes[0]), bins)
# usagesHist_nonCommu = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # commuter
# usagesHist = np.histogram(np.array(distancesInUserTypes[1]), bins)
# usagesHist_commu = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# bins = np.array(bins[1:])

# fig = plt.figure(figsize=(6, 4))
# ax = plt.subplot(1, 1, 1)

# plt.scatter(bins.tolist(), usagesHist_all.tolist(), marker='o', linewidth=1, facecolor='#41A7D8',
#     edgecolor='k', label='All')
# plt.scatter(bins.tolist(), usagesHist_commu.tolist(), marker='v', linewidth=1, facecolor='#1b7837',
#     edgecolor='k', label='Commuter')
# plt.scatter(bins.tolist(), usagesHist_nonCommu.tolist(), marker='s', linewidth=1, facecolor='#d73027',
#     edgecolor='k', label='Non-commuter')

# plt.xlim(0, maxV)
# plt.ylim(1e-6)
# plt.xticks(np.linspace(0, maxV, numInterval/10+1))
# # plt.xscale("log", nonposy='clip')
# plt.yscale("log", nonposy='clip')
# plt.xlabel(r'Travel displacement, d [km]', fontsize=12)
# plt.ylabel(r"Density, $F(d)$", fontsize=12)
# plt.legend(loc="lower left", frameon=False)

# plt.tight_layout()
# plt.savefig('../figs/displacement_userType.png', dpi=300)
# plt.savefig('../figs/displacement_userType.pdf')
# plt.close()


# # travel distance of varying types of trips
# numInterval = 100
# maxV = 100.0
# interval = maxV / numInterval
# bins = np.linspace(0, maxV, numInterval + 1)

# usagesHist = np.histogram(np.array(allTripsDistances), bins)
# usagesHist_all = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # home-work trip
# usagesHist = np.histogram(np.array(distancesInTripTypes[0]), bins)
# usagesHist_HW = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # home-other trip
# usagesHist = np.histogram(np.array(distancesInTripTypes[1]), bins)
# usagesHist_HO = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # work-other trip
# usagesHist = np.histogram(np.array(distancesInTripTypes[2]), bins)
# usagesHist_WO = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # other-other trip
# usagesHist = np.histogram(np.array(distancesInTripTypes[3]), bins)
# usagesHist_OO = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# bins = np.array(bins[1:])

# fig = plt.figure(figsize=(6, 4))
# ax = plt.subplot(1, 1, 1)

# plt.scatter(bins.tolist(), usagesHist_all.tolist(), marker='o', linewidth=1, facecolor='#41A7D8',
#     edgecolor='k', label='All')
# plt.scatter(bins.tolist(), usagesHist_HW.tolist(), marker='v', linewidth=1, facecolor='#1b7837',
#     edgecolor='k', label='Home-Work')
# plt.scatter(bins.tolist(), usagesHist_HO.tolist(), marker='s', linewidth=1, facecolor='#d73027',
#     edgecolor='k', label='Home-Other')
# plt.scatter(bins.tolist(), usagesHist_WO.tolist(), marker='s', linewidth=1, facecolor='#762a83',
#     edgecolor='k', label='Work-Other')
# plt.scatter(bins.tolist(), usagesHist_OO.tolist(), marker='s', linewidth=1, facecolor='#b35806',
#     edgecolor='k', label='Other-Other')

# plt.xlim(0, maxV)
# plt.ylim(1e-6)
# plt.xticks(np.linspace(0, maxV, numInterval/10+1))
# # plt.xscale("log", nonposy='clip')
# plt.yscale("log", nonposy='clip')
# plt.xlabel(r'Travel displacement, d [km]', fontsize=12)
# plt.ylabel(r"Density, $F(d)$", fontsize=12)
# plt.legend(loc="lower left", frameon=False)

# plt.tight_layout()
# plt.savefig('../figs/displacement_tripType.png', dpi=300)
# plt.savefig('../figs/results/displacement_tripType.pdf')
# plt.close()


# of commuters: 0.6685
# of non-commuters: 0.3315
# of no home: 0.2531


In [5]:
with open('/data6/peiyan/SH-METR/results/userLabels_light.pkl','wb') as outfile:
    pickle.dump(userLabels, outfile)